In [1]:
import random
import numpy as np
import h5py
import json
import pdb
import io
from PIL import Image
from format import pytree_display

def load_data_from_hdf5(file_list, batch_size, file_batch_size, embedding_dict):
    
  def pad_and_resize(image, target_size):
    original_size = image.size
    ratio = float(target_size) / max(original_size)
    new_size = tuple([int(x * ratio) for x in original_size])
    
    resized_image = image.resize(new_size, Image.Resampling.LANCZOS)
    new_image = Image.new("RGB", (target_size, target_size))
    new_image.paste(resized_image, ((target_size - new_size[0]) // 2, (target_size - new_size[1]) // 2))

    return new_image

  def bytes_image_to_np(image_bytes, image_size=300):
    image = Image.open(io.BytesIO(image_bytes))
    image = pad_and_resize(image, image_size)
    image_array = np.array(image)
    image_array = image_array[:,:,[2,1,0]] / 255.0
    return image_array
  
  file_list = random.sample(file_list, file_batch_size)
  num_per_file = batch_size // file_batch_size
  natural_language_embedding = []
  image = []
  arms_action = []
  
  length = np.random.randint(1, 16, batch_size)
  is_first = np.zeros((batch_size, 15), dtype=bool)
  for i in range(batch_size):
    if length[i] != 15:
      is_first[i, 15 - length[i]] = True
      
  terminate_episode = np.zeros((batch_size, 15, 3), dtype = np.int32)
  terminate_episode[:,:,1] = 1
  
  files = random.sample(file_list, file_batch_size)
  for i, file in enumerate(files):
    with h5py.File(file, 'r') as f:
      natural_language_embedding.extend([embedding_dict[f['instruction'][()].decode('utf-8')] for _ in range(num_per_file)])
      for j in range(num_per_file):
        l = length[i * num_per_file + j]
        traj_l = f['action'].shape[0]
        start = np.random.randint(0, traj_l - l + 1)
        image.append(np.pad(np.array(list(map(bytes_image_to_np, f['observations']['images']['cam_high'][start:start+l]))),
                            ((15 - l, 0), (0, 0), (0, 0), (0, 0)), 'constant'))
        arms_action.append(np.pad(np.array(f['action'][start:start+l]), ((15 - l, 0), (0, 0)), 'constant'))
  
  batch = {
    'is_first': np.array(is_first),
    'observation': {
        'image': np.array(image),
        'natural_language': np.array(natural_language_embedding)
        },
    'action': {
        'arms': np.array(arms_action),
        'terminate_episode': terminate_episode
    },
    'is_last': np.zeros((batch_size, 15), dtype=bool),
    'is_terminal': np.zeros((batch_size, 15), dtype=bool)
    }
  
  return batch
        
        
file_list = [f'/mnt/data_x2/wulingxuan/robot-transformer/data/arrange_fruits_by_size/episode_{i}.hdf5' for i in range(20)]
file_batch_size = 8
batch_size = 16
embedding_dict = json.load(open('text_embeddings.json', 'r'))

batch = load_data_from_hdf5(file_list, batch_size, file_batch_size, embedding_dict)
pytree_display(batch)

2024-05-22 00:54:52.685313: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 00:54:52.685385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 00:54:52.686335: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-22 00:54:53.536405: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


{
    "action": {
        "arms": "Shape: (16, 15, 14)",
        "terminate_episode": "Shape: (16, 15, 3)"
    },
    "is_first": "Shape: (16, 15)",
    "is_last": "Shape: (16, 15)",
    "is_terminal": "Shape: (16, 15)",
    "observation": {
        "image": "Shape: (16, 15, 300, 300, 3)",
        "natural_language": "Shape: (16, 512)"
    }
}
